Дорогой студент! 

В домашнем задании Lite вам предлагается поработать подробнее с параметрами словаря и формированием гиперпараметров нейронной сети. Создайте 9 нейросетей с различными гиперпараметрами (см. пунтк 2 и 3)

 Для этого необходимо:

  1. Воссоздать ноутбук, аналогичный ноутбуку теоретической части, загрузив при этом необходимую нам базу (код уже доступен в ноутбуке).

  2. Задать в ноутбуке следующие параметры для размера словаря, ширины окна и шага:

    - Размер словаря - от 10000 до 20000 (выбрать меньшее значение диапазона, если будет перегрузка ОЗУ и перезапуск подключения к Colaboratory)
    - Ширина окна - от 1000 до 2000
    - Шаг - от 100 до 500 (на обучение лучше влияет наименьший шаг, но это может перегрузить ОЗУ).

  3. Создать архитектуру сети и задать гиперпараметры. Можно воспользоваться шаблоном:
  
   - Добавьте модель прямого распространения **Sequential()**
   - Добавьте один или несколько полносвязных (**Dense**) слоёв
   - Добавьте слои **Dropout()** и **BatchNormalization()**
   - Добавьте выходной полносвязный слой с количеством нейронов, соответствующим количеству классов (число писателей)
  
   Напомним, что точность сети можно проверить по значению показателя 'val_accuracy' на конце каждой эпохи. 
   

## Подготовка

**Шаг 1**. Загрузка бибилиотек и базы данных.

Подключаем необходимые библиотеки и модули:

In [ ]:
# Работа с массивами данных
import numpy as np 

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation 

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Матрица ошибок классификатора
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Работа с google диском
from google.colab import drive

# Загрузка датасетов из облака google
import gdown

# Функции операционной системы
import os

# Регулярные выражения
import re

Потом загружаем датасет с текстами писателей в виде архива и распаковываем его в папку:

In [ ]:
# Загрузка датасета из облака
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l7/writers.zip', None, quiet=True)

# Распаковка архива в папку writers
!unzip -qo writers.zip -d writers/

# Просмотр содержимого папки
!ls writers

'(Булгаков) Обучающая_5 вместе.txt'
'(Булгаков) Тестовая_2 вместе.txt'
'(Клиффорд_Саймак) Обучающая_5 вместе.txt'
'(Клиффорд_Саймак) Тестовая_2 вместе.txt'
'(Макс Фрай) Обучающая_5 вместе.txt'
'(Макс Фрай) Тестовая_2 вместе.txt'
'(О. Генри) Обучающая_50 вместе.txt'
'(О. Генри) Тестовая_20 вместе.txt'
'(Рэй Брэдберри) Обучающая_22 вместе.txt'
'(Рэй Брэдберри) Тестовая_8 вместе.txt'
'(Стругацкие) Обучающая_5 вместе.txt'
'(Стругацкие) Тестовая_2 вместе.txt'


Сохраните в отдельных переменных:

   - Имя для папки с текстами
   - Название выборки "обучающая"
   - Название выборки "тестовая"

In [ ]:
# Папка с текстовыми файлами
FILE_DIR  = 'writers'

# Признак обучающей выборки в имени файла                     
SIG_TRAIN = 'обучающая'

# Признак тестовой выборки в имени файла                   
SIG_TEST  = 'тестовая'                    

**Шаг 2.** Преобразование базы данных для обучения.

Добавляем имена всех писателей в список классов.

Воспроизводим блок кода, где преобразуем тексты в одну длинную строку и разбиваем их отдельно в списки для каждого класса и выборки.

In [ ]:
# В список добавляются имена классов
CLASS_LIST = []

# Здесь сохраняются тексты для обучения сети
text_train = []

# А здесь для проверки точности сети
text_test = []
 
# цикл для итерации по каждому имени текста в общей папке
for file_name in os.listdir(FILE_DIR):

    # Выделение имени класса и типа выборки из имени файла
    m = re.match('\((.+)\) (\S+)_', file_name)

    # Если выделение получилось, то файл обрабатывается
    if m:

        # отдельно берём имя класса (автора)
        class_name = m[1]

        # отдельно - имя выборки lower не с заглавной буквы
        subset_name = m[2].lower()

        # Проверка типа выборки по названию в имени файла
        is_train = SIG_TRAIN in subset_name
        is_test = SIG_TEST in subset_name

        # Если тип выборки обучающая или тестовая - файл обрабатывается
        if is_train or is_test:

            # Добавление нового класса, если его еще нет в списке
            if class_name not in CLASS_LIST:

                # Выводится информационное сообщение о добавлении названия класса
                CLASS_LIST.append(class_name)

                # Инициализация соответствующих классу строк текста
                text_train.append('')
                text_test.append('')

            # Поиск индекса класса для добавления содержимого файла в выборку
            cls = CLASS_LIST.index(class_name)

            # Выводится информационное сообщение о добавлении класса в список классов и текста к выборке
            print(f'Добавление файла "{file_name}" в класс "{CLASS_LIST[cls]}", {subset_name} выборка.')

            # оператор with - безопасное чтение каждого файла с текстом
            with open(f'{FILE_DIR}/{file_name}', 'r') as f:

                # Загрузка содержимого файла в строку
                text = f.read()

            # Определение выборки, куда будет добавлено содержимое
            subset = text_train if is_train else text_test

            # Добавление текста к соответствующей выборке класса. Концы строк заменяются на пробел
            subset[cls] += ' ' + text.replace('\n', ' ')
            
# Определим кол-во классов
CLASS_COUNT = len(CLASS_LIST)

Добавление файла "(О. Генри) Тестовая_20 вместе.txt" в класс "О. Генри", тестовая выборка.
Добавление файла "(Клиффорд_Саймак) Обучающая_5 вместе.txt" в класс "Клиффорд_Саймак", обучающая выборка.
Добавление файла "(Клиффорд_Саймак) Тестовая_2 вместе.txt" в класс "Клиффорд_Саймак", тестовая выборка.
Добавление файла "(Рэй Брэдберри) Обучающая_22 вместе.txt" в класс "Рэй Брэдберри", обучающая выборка.
Добавление файла "(Булгаков) Тестовая_2 вместе.txt" в класс "Булгаков", тестовая выборка.
Добавление файла "(Рэй Брэдберри) Тестовая_8 вместе.txt" в класс "Рэй Брэдберри", тестовая выборка.
Добавление файла "(Макс Фрай) Обучающая_5 вместе.txt" в класс "Макс Фрай", обучающая выборка.
Добавление файла "(Стругацкие) Тестовая_2 вместе.txt" в класс "Стругацкие", тестовая выборка.
Добавление файла "(О. Генри) Обучающая_50 вместе.txt" в класс "О. Генри", обучающая выборка.
Добавление файла "(Макс Фрай) Тестовая_2 вместе.txt" в класс "Макс Фрай", тестовая выборка.
Добавление файла "(Стругацкие) Об

Подготовьте текстовую инфрмацию для обработки Токенайзером. Для этого сохраните в подходящих именах переменных следующие параметры:

   - Объем словаря для токенизатора
   - Длина отрезка текста (окна) в словах
   - Шаг смещения окна по тексту для деления на векторы

И затем вызовите сам Токенайзер, передав в его параметры эти переменные и оставшиеся значения для преобразования.

In [ ]:
# Объем словаря для токенизатора
VOCAB_SIZE = 15000

# Длина отрезка текста (окна) в словах                        
WIN_SIZE   = 3000

# Шаг окна разбиения текста на векторы                         
WIN_HOP    = 300                          

# Токенайзер из Keras для разбиения текста и построения частотного словаря
tokenizer = Tokenizer(num_words=VOCAB_SIZE, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', 
                          lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)

Отдельно обучите сам Токенайзер на выборке текстов для обучения сети:


In [ ]:
# Получаем словарь частотности 
tokenizer.fit_on_texts(text_train)

Преобразуйте обучающие и проверочные тексты в последовательность индексов согласно частотному словарю Токенайзера:

In [ ]:
seq_train = tokenizer.texts_to_sequences(text_train)
seq_test = tokenizer.texts_to_sequences(text_test)

**Шаг 3.** Создание функций для формирования выборок. Формирование выборок.

Воспроизведите блок кода с двумя функциями:

  1. Функция деления последовательности индексов на отрезки скользящим окном

In [ ]:
def split_sequence(sequence,   # Последовательность индексов
                   win_size,   # Размер окна для деления на примеры
                   hop):       # Шаг окна

    # Последовательность разбивается на части до последнего полного окна
    return [sequence[i:i + win_size] for i in range(0, len(sequence) - win_size + 1, hop)]

 2. Функция формирования выборок из индексов и соответствующих классам меток в формате One Hot Encoding

In [ ]:
def vectorize_sequence(seq_list,    # Список последовательностей индексов 
                       win_size,    # Размер окна для деления на примеры
                       hop):        # Шаг окна

    # В списке последовательности следуют в порядке их классов (их кол-во сповпадает с кол-вом классов)
    class_count = len(seq_list)

    # Списки для исходных векторов и категориальных меток класса
    x, y = [], []

    # Для каждого класса:
    for cls in range(class_count):

        # Разбиение последовательности класса cls на отрезки
        vectors = split_sequence(seq_list[cls], win_size, hop)

        # Добавление отрезков в выборку

        x += vectors
        
        # Для всех отрезков класса cls добавление меток класса в виде OHE
        y += [utils.to_categorical(cls, class_count)] * len(vectors)

    # Возврат результатов как numpy-массивов
    return np.array(x), np.array(y)

Сформируйте выборки (x_train, y_train) и (x_test, y_test) с помощью функций, воспроизведённых в ячейке выше:

In [ ]:
# Формирование обучающей выборки
x_train, y_train = vectorize_sequence(seq_train, WIN_SIZE, WIN_HOP) 
# Формирование тестовой выборки
x_test, y_test = vectorize_sequence(seq_test, WIN_SIZE, WIN_HOP)

# Проверка формы сформированных данных
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5842, 3000) (5842, 6)
(2190, 3000) (2190, 6)


Сформируем выборки индексов слов в виде матриц Bag Of Words методом  **sequences_to_matrix( )**. 

In [ ]:
# На входе .sequences_to_matrix() ожидает список, .tolist() выполняет преобразование к типу данных 'список'
x_train_01 = tokenizer.sequences_to_matrix(x_train.tolist())
x_test_01 = tokenizer.sequences_to_matrix(x_test.tolist())

# Выводим форму обучающей выборки в виде матрицы Bag of Words
print(x_train_01.shape)       
# Выводим фрагмент отрезка обучающего текста в виде Bag of Words
print(x_train_01[0][0:100])

(5842, 15000)
[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 1.]


**Шаг 4.** Создание архитектуры нейронной сети.

Создайте несложную архитектуру нейронной сети. После этого произведите компиляцию модели с соответствующими параметрами:

In [ ]:
model_BoW = Sequential()                                            # Создание последовательной модели нейросети
model_BoW.add(Dense(184, input_dim=VOCAB_SIZE, activation="relu"))  # Первый полносвязный слой
model_BoW.add(Dense(86, activation="relu"))                         # Второй полносвязный слой
model_BoW.add(Dense(33, activation="relu"))                         # Третий полносвязный слой
model_BoW.add(BatchNormalization())                                 # Слой пакетной нормализации
model_BoW.add(Dropout(0.1))                                         # Слой регуляризации Dropout
model_BoW.add(Dense(CLASS_COUNT, activation='sigmoid'))             # Выходной полносвязный слой

model_BoW.compile(optimizer='adam',                                 # Компиляция модели для обучения на данных вида Bag of Words
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

Обучите модель нейронной сети и посмотрите на результаты точности на проверочной выборке:

In [ ]:
# Обучение сети с помощью функции fit()
history = model_BoW.fit(x_train_01,                            # Обучающая выборка Bag of Words
                        y_train,                               # Метки классов обучающей выборки
                        epochs=20,                             # Количество эпох
                        batch_size=32,                         # Размер подвыборки для одного шага по данным на эпохе
                        validation_data=(x_test_01, y_test))   # Проверочная выборка и метки классов проверочной выборки

Epoch 1/20
183/183 [==============================] - 6s 25ms/step - loss: 0.1396 - accuracy: 0.9801 - val_loss: 0.4498 - val_accuracy: 0.9009
Epoch 2/20
183/183 [==============================] - 4s 23ms/step - loss: 0.0134 - accuracy: 1.0000 - val_loss: 0.3672 - val_accuracy: 0.9205
Epoch 3/20
183/183 [==============================] - 4s 23ms/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 0.3293 - val_accuracy: 0.9233
Epoch 4/20
183/183 [==============================] - 4s 23ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.2880 - val_accuracy: 0.9306
Epoch 5/20
183/183 [==============================] - 4s 23ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.2657 - val_accuracy: 0.9406
Epoch 6/20
183/183 [==============================] - 4s 23ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.2744 - val_accuracy: 0.9301
Epoch 7/20
183/183 [==============================] - 4s 23ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.2601 - val_accuracy: 0.9274

Запишите результаты в таблицу:

In [ ]:
# Ваше задание здесь

In [ ]:
model_BoW1 = Sequential()                                            # Создание последовательной модели нейросети
model_BoW1.add(Dense(256, input_dim=VOCAB_SIZE, activation="relu"))  # Первый полносвязный слой
model_BoW1.add(Dense(85, activation="relu"))                         # Второй полносвязный слой
model_BoW1.add(Dense(21, activation="relu"))                         # Третий полносвязный слой
model_BoW1.add(BatchNormalization())                                 # Слой пакетной нормализации

model_BoW1.add(Dense(CLASS_COUNT, activation='sigmoid'))             # Выходной полносвязный слой





model_BoW1.compile(optimizer='adam',                                 # Компиляция модели для обучения на данных вида Bag of Words
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
# Обучение сети с помощью функции fit()
history = model_BoW1.fit(x_train_01,                            # Обучающая выборка Bag of Words
                        y_train,                               # Метки классов обучающей выборки
                        epochs=20,                             # Количество эпох
                        batch_size=32,                         # Размер подвыборки для одного шага по данным на эпохе
                        validation_data=(x_test_01, y_test))   # Проверочная выборка и метки классов проверочной выборки

Epoch 1/20
183/183 [==============================] - 7s 32ms/step - loss: 0.1250 - accuracy: 0.9841 - val_loss: 0.5065 - val_accuracy: 0.8342
Epoch 2/20
183/183 [==============================] - 6s 31ms/step - loss: 0.0122 - accuracy: 1.0000 - val_loss: 0.3723 - val_accuracy: 0.9187
Epoch 3/20
183/183 [==============================] - 6s 30ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.3160 - val_accuracy: 0.9466
Epoch 4/20
183/183 [==============================] - 6s 30ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.2969 - val_accuracy: 0.9470
Epoch 5/20
183/183 [==============================] - 6s 30ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.2911 - val_accuracy: 0.9388
Epoch 6/20
183/183 [==============================] - 6s 31ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.2705 - val_accuracy: 0.9461
Epoch 7/20
183/183 [==============================] - 6s 30ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.2626 - val_accuracy: 0.9466

model_BoW1 

In [ ]:
# Объем словаря для токенизатора
VOCAB_SIZE = 10000

# Длина отрезка текста (окна) в словах                        
WIN_SIZE   = 1000

# Шаг окна разбиения текста на векторы                         
WIN_HOP    = 100                          

# Токенайзер из Keras для разбиения текста и построения частотного словаря
tokenizer = Tokenizer(num_words=VOCAB_SIZE, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', 
                          lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)

In [ ]:
# Получаем словарь частотности 
tokenizer.fit_on_texts(text_train)

In [ ]:
#Преобразуйте обучающие и проверочные тексты в последовательность индексов согласно частотному словарю Токенайзера:


In [ ]:
seq_train = tokenizer.texts_to_sequences(text_train)
seq_test = tokenizer.texts_to_sequences(text_test)

In [ ]:
def split_sequence(sequence,   # Последовательность индексов
                   win_size,   # Размер окна для деления на примеры
                   hop):       # Шаг окна

    # Последовательность разбивается на части до последнего полного окна
    return [sequence[i:i + win_size] for i in range(0, len(sequence) - win_size + 1, hop)]

In [ ]:
def vectorize_sequence(seq_list,    # Список последовательностей индексов 
                       win_size,    # Размер окна для деления на примеры
                       hop):        # Шаг окна

    # В списке последовательности следуют в порядке их классов (их кол-во сповпадает с кол-вом классов)
    class_count = len(seq_list)

    # Списки для исходных векторов и категориальных меток класса
    x, y = [], []

    # Для каждого класса:
    for cls in range(class_count):

        # Разбиение последовательности класса cls на отрезки
        vectors = split_sequence(seq_list[cls], win_size, hop)

        # Добавление отрезков в выборку

        x += vectors
        
        # Для всех отрезков класса cls добавление меток класса в виде OHE
        y += [utils.to_categorical(cls, class_count)] * len(vectors)

    # Возврат результатов как numpy-массивов
    return np.array(x), np.array(y)

In [ ]:
# Формирование обучающей выборки
x_train, y_train = vectorize_sequence(seq_train, WIN_SIZE, WIN_HOP) 
# Формирование тестовой выборки
x_test, y_test = vectorize_sequence(seq_test, WIN_SIZE, WIN_HOP)

# Проверка формы сформированных данных
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(17640, 1000) (17640, 6)
(6686, 1000) (6686, 6)


In [ ]:
# На входе .sequences_to_matrix() ожидает список, .tolist() выполняет преобразование к типу данных 'список'
x_train_01 = tokenizer.sequences_to_matrix(x_train.tolist())
x_test_01 = tokenizer.sequences_to_matrix(x_test.tolist())

# Выводим форму обучающей выборки в виде матрицы Bag of Words
print(x_train_01.shape)       
# Выводим фрагмент отрезка обучающего текста в виде Bag of Words
print(x_train_01[0][0:100])

(17640, 10000)
[0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0.
 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0.
 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1.
 0. 0. 1. 1.]


In [ ]:
model_BoW = Sequential()                                            # Создание последовательной модели нейросети
model_BoW.add(Dense(184, input_dim=VOCAB_SIZE, activation="relu"))  # Первый полносвязный слой
model_BoW.add(Dense(86, activation="relu"))                         # Второй полносвязный слой
model_BoW.add(Dense(33, activation="relu"))                         # Третий полносвязный слой
model_BoW.add(BatchNormalization())                                 # Слой пакетной нормализации
model_BoW.add(Dropout(0.1))                                         # Слой регуляризации Dropout
model_BoW.add(Dense(CLASS_COUNT, activation='sigmoid'))             # Выходной полносвязный слой

model_BoW.compile(optimizer='adam',                                 # Компиляция модели для обучения на данных вида Bag of Words
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
# Обучение сети с помощью функции fit()
history = model_BoW.fit(x_train_01,                            # Обучающая выборка Bag of Words
                        y_train,                               # Метки классов обучающей выборки
                        epochs=20,                             # Количество эпох
                        batch_size=32,                         # Размер подвыборки для одного шага по данным на эпохе
                        validation_data=(x_test_01, y_test))   # Проверочная выборка и метки классов проверочной выборки

Epoch 1/20
552/552 [==============================] - 11s 18ms/step - loss: 0.0600 - accuracy: 0.9871 - val_loss: 0.4312 - val_accuracy: 0.8525
Epoch 2/20
552/552 [==============================] - 9s 17ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.4092 - val_accuracy: 0.8576
Epoch 3/20
552/552 [==============================] - 11s 19ms/step - loss: 8.4706e-04 - accuracy: 1.0000 - val_loss: 0.3871 - val_accuracy: 0.8630
Epoch 4/20
552/552 [==============================] - 9s 17ms/step - loss: 4.1004e-04 - accuracy: 1.0000 - val_loss: 0.3897 - val_accuracy: 0.8570
Epoch 5/20
552/552 [==============================] - 9s 17ms/step - loss: 2.7310e-04 - accuracy: 1.0000 - val_loss: 0.4063 - val_accuracy: 0.8527
Epoch 6/20
552/552 [==============================] - 9s 17ms/step - loss: 1.5924e-04 - accuracy: 1.0000 - val_loss: 0.3776 - val_accuracy: 0.8648
Epoch 7/20
552/552 [==============================] - 9s 17ms/step - loss: 1.1405e-04 - accuracy: 1.0000 - val_loss: 0.3827 

In [ ]:
# Для формирования таблицы
import pandas as pd

In [ ]:
df = pd.DataFrame({'VOCAB_SIZE':[15000, 15000, 10000], 'WIN_SIZE':[3000, 3000, 1000], 'WIN_HOP':[300, 300, 100], 'Dropout':[0.1, 0, 0.1],
                   'Dense':[(184,86,33), (256,85,21), (184,86,33)], 'val_accuracy':[0.92, 0.95, 0.81]})

In [ ]:
print(df) 

   VOCAB_SIZE  WIN_SIZE  WIN_HOP  Dropout          Dense  val_accuracy
0       15000      3000      300      0.1  (184, 86, 33)          0.92
1       15000      3000      300      0.0  (256, 85, 21)          0.95
2       10000      1000      100      0.1  (184, 86, 33)          0.81
